In [1]:
import os
import time 
import datetime

def print_table(data: dict[str, list], precision: int = 4):
    if not data:
        print("(empty)")
        return

    # Determine number of rows and ensure index column exists and is first
    n_rows = len(next(iter(data.values())))
    if "index" not in data or len(data["index"]) != n_rows:
        data["index"] = list(range(1, n_rows + 1))
    headers = ["index"] + [h for h in data.keys() if h != "index"]

    # Numeric format
    fmt = f"{{:.{precision}f}}"

    # Detect string columns: if any value is a string, treat entire column as strings
    is_string_col = {
        h: any(isinstance(v, str) for v in data[h])
        for h in headers
    }

    # Compute column widths
    col_widths = []
    for h in headers:
        values = data[h]
        if is_string_col[h]:
            max_val_len = max(len(str(v)) for v in values)
        else:
            max_val_len = max(len(fmt.format(v)) for v in values)
        col_widths.append(max(len(h), max_val_len))

    # Build header row (strings left-aligned, numbers right-aligned)
    header_cells = []
    for h, w in zip(headers, col_widths):
        header_cells.append(f"{h:<{w}}" if is_string_col[h] else f"{h:>{w}}")
    header_row = " | ".join(header_cells)
    separator = "-+-".join("-" * w for w in col_widths)

    # Print table
    print(header_row)
    print(separator)
    for i in range(n_rows):
        row_cells = []
        for h, w in zip(headers, col_widths):
            v = data[h][i]
            if is_string_col[h]:
                row_cells.append(str(v).ljust(w))
            else:
                row_cells.append(fmt.format(v).rjust(w))
        print(" | ".join(row_cells))

In [2]:
def dt_to_intkey(date_str: str, time_str: str) -> float:
    dt = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y_%m_%d %H_%M_%S")
    return dt.timestamp()

In [7]:
data_dir = "../../logs/full_runs/2026_01_18_assorted_dataset_test/results/"
res_files_unfiltered = [f for f in os.listdir(data_dir) if f.endswith(".log")]
res_files_unfiltered.sort()
# print(*res_files_unfiltered, sep="\n")
dates, times = zip(*(s.split("__")[:2] for s in res_files_unfiltered))
creation_times = {
    # key is date_time string converted to int/float for sorting
    index: dt_to_intkey(date, time)
    for index, (date, time) in enumerate(zip(dates, times))
}
only_after = dt_to_intkey("2025_10_30", "00_00_00")
res_files = [
    res_files_unfiltered[index]
    for index, creation_time in creation_times.items()
    if creation_time >= only_after
]
# print(*res_files, sep="\n")
print(len(res_files), "files found after filtering.")

14 files found after filtering.


In [11]:
# Example log lines we are looking for:
# Using color space YCoCg with bitdepths [8, 9, 9]
# Using image ARM: True
# Using encoder gain: 64
# Using multi-region image ARM: True
# Using color regression: False
# Total training iterations: 142200
# Total MAC per pixel: 1694.3125
# Final results after quantization: Loss: 3.278064727783203, Rate NN: 0.11986033121744792, Rate Latent: 0.1637668013572693, Rate Img: 2.9944374561309814
# Rate Img bistream: 7.751302083333333

using_color_space = ""
using_image_arm = False
using_encoder_gain = 0
using_multi_region_image_arm = False
using_color_regression = False
total_training_iterations = 0
total_mac_per_pixel = 0.0
final_results = []
rate_img_bistream = []

with open(os.path.join(data_dir, res_files[0]), "r") as infile:
    lines = infile.readlines()    
    for line in lines:
        if line.startswith("Using color space"):
            using_color_space = line[len("Using color space") :].strip().split(" with bitdepths")[0]
        if line.startswith("Using image ARM:"):
            using_image_arm = line[len("Using image ARM:"):].strip() == "True"
        if line.startswith("Using encoder gain:"):
            using_encoder_gain = int(line[len("Using encoder gain:"):].strip())
        if line.startswith("Using multi-region image ARM:"):
            using_multi_region_image_arm = line[len("Using multi-region image ARM:"):].strip() == "True"
        if line.startswith("Using color regression:"):
            using_color_regression = line[len("Using color regression:"):].strip() == "True"
        if line.startswith("Total training iterations:"):
            total_training_iterations = int(line[len("Total training iterations:"):].strip())
        if line.startswith("Total MAC per pixel:"):
            total_mac_per_pixel = float(line[len("Total MAC per pixel:"):].strip())

    if using_color_space == "":
        print(f"Could not determine color space for file {res_files[0]}, skipping.")
        raise ValueError("No color space found.")
print("Using color space:", using_color_space)
print("Using Image ARM:", using_image_arm)
print("Using encoder gain:", using_encoder_gain)
print("Using multi-region image ARM:", using_multi_region_image_arm)
print("Using color regression:", using_color_regression)
print("Total training iterations:", total_training_iterations)
print("Total MAC per pixel:", total_mac_per_pixel)


for f in res_files:
    with open(os.path.join(data_dir, f), "r") as infile:
        lines = infile.readlines()   

        for line in lines:
            if line.startswith("Rate Img bistream:"):
                rate_img_bistream.append(float(line[len("Rate Img bistream:"):].strip()))
            if "Best loss for module upsampling:" in line:
                parts = line[len("Best loss for module upsampling:") :].strip().split(", ")
                # im_index = int(f.split("_")[-1][len("kodim") :][: -len(".log")]) - 1
                final_results.append({})
                final_results[-1]["Im_name"] = f.split("_")[-1][:6]
                for part in parts:
                    key, value = part.split(": ")
                    final_results[-1][key] = float(value)

# print("Rate Img bistreams:", rate_img_bistream)
# print("Final results collected:", final_results)

def list_dict_to_dict_list(
    lst: list[dict[str, float]],
) -> dict[str, list[float]]:
    if not lst:
        return {}
    keys = lst[0].keys()
    dict_list = {key: [] for key in keys}
    for d in lst:
        for key in keys:
            dict_list[key].append(d.get(key, 0.0))
    return dict_list

results_table = list_dict_to_dict_list(final_results)
results_table["Rate Img bistream"] = rate_img_bistream
print_table(results_table)

for key in results_table.keys():
    # results_for_key = [d[0] for d in results_table[key]]
    # results_for_key = list_dict_to_dict_list(results_for_key)

    # print(f"{key}, Using Image ARM: {using_image_arm} Results:")
    # for i in range(len(results_for_key['Loss'])):
    #     results_for_key['Loss'][i] = results_for_key['Loss'][i] - results_for_key['Rate NN'][i]
    #     results_for_key['Rate NN'][i] = results_for_key['Rate NN'][i] #/ (512 * 768 * 3)
    #     results_for_key['Loss'][i] = results_for_key['Loss'][i] + results_for_key['Rate NN'][i]
    # print_table(results_for_key)
    # for key, values in results_for_key.items():
    avg = sum(results_table[key]) / len(results_table[key]) if (results_table[key] and isinstance(results_table[key][0], (int, float))) else 0
    print(f"{key}: {avg:.4f}")

Using color space: YCoCg
Using Image ARM: True
Using encoder gain: 64
Using multi-region image ARM: False
Using color regression: False
Total training iterations: 2300
Total MAC per pixel: 1696.40234375
  index | Im_name |   Loss | Rate NN | Rate Latent | Rate Img | Rate Img bistream
--------+---------+--------+---------+-------------+----------+------------------
 1.0000 | 591e85  | 3.4748 |  0.0820 |      0.1208 |   3.2720 |            3.2871
 2.0000 | 8019e8  | 3.0075 |  0.0735 |      0.3680 |   2.5661 |            2.5977
 3.0000 | 0808.l  | 4.7040 |  0.0675 |      0.2804 |   4.3562 |            4.3870
 4.0000 | 0879.l  | 3.4751 |  0.0736 |      0.2041 |   3.1974 |            3.2153
 5.0000 | 4.log   | 3.6736 |  0.0727 |      0.1432 |   3.4577 |            3.4635
 6.0000 | 8.log   | 3.8520 |  0.0790 |      0.2097 |   3.5633 |            3.5685
 7.0000 | 13.log  | 4.4871 |  0.0671 |      0.1992 |   4.2208 |            4.2459
 8.0000 | 7.log   | 4.3580 |  0.0616 |      0.1747 |   4.12